# Analítica y Ciencia de Datos

## CIDE - Otoño 2015

### Solución Tarea 3: Pandas


## Primer Ejercicio

* **Parte a**: El error se obtiene porque el DataFrame `mat_mun` tiene 522 filas, una por cada código de municipio incluido.  En el loop, cuando llegamos al último municipio y se ejecuta la línea:

> `ind_sig = mat_mun.Indice.ix[int(preind + 1)]`

obtenemos un error cuando Pandas intenta acceder al índice *523*.

* **Parte b**: La solución es muy simple.  Con un `if-else` basta:

> ```
> if int(preind + 1) < mat_mun.shape[0]:
      ind_sig = mat_mun.Indice.ix[int(preind + 1)]
>```


## Segundo Ejercicio

#### Parte a:

In [34]:
import pandas as pd
import numpy as np
np.random.seed(10)
newdf = pd.DataFrame(np.random.randn(20,2), columns=['columna1', 'columna2'])
newdf.head()

,columna1,columna2
0,1.331587,0.715279
1,-1.545400,-0.008384
2,0.621336,-0.720086
3,0.265512,0.108549
4,0.004291,-0.174600


#### Parte b

In [35]:
# Una forma es con la función .ix
newdf.columna1.ix[1:4] = np.nan
newdf.columna1.ix[7:9] = np.nan
newdf.columna1.ix[15:19] = np.nan
newdf.head(6)

,columna1,columna2
0,1.331587,0.715279
1,NaN,-0.008384
2,NaN,-0.720086
3,NaN,0.108549
4,NaN,-0.174600
5,0.433026,1.203037


In [72]:
# Alternativamente se puede hacer con la función .loc y de un sólo paso
sel_arr = np.concatenate((np.arange(1,5), np.arange(7,10), np.arange(15,20)),axis=0)
newdf.columna1.loc[sel_arr] = np.nan
newdf.head(7)

,columna1,columna2
0,1.331587,0.715279
1,NaN,-0.008384
2,NaN,-0.720086
3,NaN,0.108549
4,NaN,-0.174600
5,0.433026,1.203037
6,-0.965066,1.028274


#### Parte c

In [73]:
ind_not_nan = newdf[newdf.columna1.isnull()==False].index
#ind_not_nan
for i in range(ind_not_nan.shape[0]):
    # seleccionemos el actual:
    ind_actual = ind_not_nan[i]
    # seleccionemos el siguiente:
    # para evitarnos un else: podemos incializarlo: con el default
    ind_siguiente = newdf.shape[0]
    if i+1 < ind_not_nan.shape[0]:
        ind_siguiente = ind_not_nan[i+1]-1
    #else:
    #    ind_siguiente = newdf.shape[0]

    # listo para copiar:
    newdf.columna1.ix[ind_actual:ind_siguiente] = newdf.columna1.ix[ind_actual]    
newdf

,columna1,columna2
0,1.331587,0.715279
1,1.331587,-0.008384
2,1.331587,-0.720086
3,1.331587,0.108549
4,1.331587,-0.174600
5,0.433026,1.203037
6,-0.965066,1.028274
7,-0.965066,0.445138
8,-0.965066,0.135137
9,-0.965066,-1.079805


In [76]:
%timeit exec In[73]

100 loops, best of 3: 5.3 ms per loop


#### Parte d: otro método

El método más natural es hacer un loop sobre las celdas, verificar si hay un NaN.  Si si, copio la celda anterior, si no, sigo.  Por supuesto, este método es muy costoso en términos computacionales.

En este caso la matriz es tan pequeña que este segundo método es menos costos (2.08ms vs. 5.3ms).  Las ganancias se ven cuando la matriz es mucho más grande.  El loop de este método se repite $N$ veces, donde $N$ es el tamaño del dataframe.  El loop del otro método se repite $M$ veces, donde $M$ es el número de elementos no nulos.  La ganancia se da cuando $M<<N$ como en el caso de los datos utilizados en las Notas de Clase.

In [66]:
# Alternativamente se puede hacer con la función .loc y de un sólo paso
sel_arr = np.concatenate((np.arange(1,5), np.arange(7,10), np.arange(15,20)),axis=0)
newdf.columna1.loc[sel_arr] = np.nan
newdf.head(7)

,columna1,columna2
0,1.331587,0.715279
1,NaN,-0.008384
2,NaN,-0.720086
3,NaN,0.108549
4,NaN,-0.174600
5,0.433026,1.203037
6,-0.965066,1.028274


In [69]:
for i in range(newdf.shape[0]):
    if newdf.columna1.isnull().ix[i]==True:
        newdf.columna1.ix[i] =  newdf.columna1.ix[i-1]
newdf        

,columna1,columna2
0,1.331587,0.715279
1,1.331587,-0.008384
2,1.331587,-0.720086
3,1.331587,0.108549
4,1.331587,-0.174600
5,0.433026,1.203037
6,-0.965066,1.028274
7,-0.965066,0.445138
8,-0.965066,0.135137
9,-0.965066,-1.079805


In [78]:
%timeit exec In[69]

100 loops, best of 3: 2.3 ms per loop


## Ejercicio tres: merge, importar y exportar datos

In [83]:
# Primer importemos los nombres de las hojas:
archivo = pd.ExcelFile('data_tarea3.xlsx')
hojas   = archivo.sheet_names
print hojas

[u'enero', u'febrero', u'marzo', u'abril']


In [97]:
# vamos a hacer un loop sobre las hojas
# La primera hoja inicializa todo:
counter = 0
for h in hojas:
    if counter==0: # la primera hoja tiene un tratamiento especial, sin merge:
        data = pd.read_excel('data_tarea3.xlsx',h)
        # lo único que tenemos que hacer es cambiar el nombre de la columna VENTAS por el mes
        data.rename(columns={'ventas':h}, inplace=True)
    else: 
        # para las siguientes tenemos que importar, cambiar el nombre y hacer un merge
        # importemos
        predata = pd.read_excel('data_tarea3.xlsx',h)
        # cambio de nombre
        predata.rename(columns={'ventas':h}, inplace=True)
        # listo para el merge
        data = data.merge(predata, on='id_tienda', how='outer')
        # actualicemos el contador
    counter +=1
# terminamos
# limpiemos los NaNs... son ventas cero
data.fillna(0, inplace= True)
data
# exportemos los datos
data.to_excel('ventas_enero_abril.xlsx')